# Standalone AssistantAgent

AG2 supports running `AssistantAgent` as a standalone agent with the ability to execute simple tasks without the need for interacting with other agents.

To enable our assistant agent to surf the web, we will use `BrowserUseTool` fow which we need to install the browser-use optional dependency and [playwright](https://playwright.dev/python/docs/intro)

````{=mdx}
:::info Requirements
Install `ag2`:
```bash
pip install ag2[browser-use]
playwright install
```
````


In [ ]:
import autogen
from autogen import AssistantAgent
from autogen.tools.experimental.browser_use.browser_use import BrowserUseTool

In [2]:
import nest_asyncio

nest_asyncio.apply()

## Set your API Endpoint

The [`config_list_from_json`](https://docs.ag2.ai/reference/autogen/config_list_from_json#config-list-from-json) function loads a list of configurations from an environment variable or a json file.

In [3]:
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "tags": ["gpt-4o-mini"],
    },
)

llm_config = {
    "timeout": 600,
    "config_list": config_list,
    "temperature": 0.8,
}

# Configure your assistant agent

Here we will configure two assistant agents:
1. x_assistant, tasked with exploring the trending topics on X (Formally Twitter)
2. arxiv_researcher, tasked with discovery of papers that allign with the hot topic of the day

In [ ]:
x_assistant = AssistantAgent(name="x_assistant", llm_config=llm_config)

arxiv_researcher = AssistantAgent(name="arxiv", llm_config=llm_config)

browser_use_tool = BrowserUseTool(
    llm_config=llm_config,
)

## Running the assistant agents

Lets run our x_assistant to discover the hot topic of the day
To be able to do this let's give our assistant the capability to browse the web using a `BrowserUseTool`

In [ ]:
hot_topic_res = x_assistant.run(
    "Find out today's hot topic and an influencer who is talking about it on X using a web search",
    tools=browser_use_tool,
)

print(hot_topic_res)

After discovering the hot topic, lets run the discovery of papers that allign with the topic

In [ ]:
paper_abstract = arxiv_researcher.run(
    "Get the abstract of a relevant paper based on " + hot_topic_res,
)

print(paper_abstract)

Now, Lets create a twitter post using our x_assistant

In [ ]:
# Secneario 1. This task requires x_assistant's past state
post = x_assistant.run(
    "Create an X post based on the hot topic and this " + paper_abstract + "and mention the influencer",
)

print(post)

Finally, lets ask our x_assistant who should we follow on twitter

In [ ]:
# Scenario 2.  Doing another task that does not require history or past state

influencer_choice = x_assistant.run(
    "Find a influencer I should follow on Twitter by searching the web",
    clear_history=True,
    tools=browser_use_tool,
)

print(influencer_choice)